## **Imports**

In [1]:
import os
import joblib
import numpy as np
import pandas as pd


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

## **Paths & Global Variables**

In [2]:
ROOT_PATH = r"C:\Users\mario\OneDrive\Documents\Work\Side Hustles\Kaggle\titanic"

import sys

sys.path.append(ROOT_PATH)

from titanic.config import PROCESSED_DATA_DIR, MODELS_DIR

2025-02-21 11:19:17.223 | INFO     | titanic.config:<module>:9 - PROJ_ROOT path is: C:\Users\mario\OneDrive\Documents\Work\Side Hustles\Kaggle\titanic


## **Reading Data**

In [3]:
os.chdir(PROCESSED_DATA_DIR)

X_test = pd.read_parquet("X_test.parquet")
y_test = pd.read_parquet("y_test.parquet")

os.chdir(MODELS_DIR)
logreg_model = joblib.load("log_reg-model.joblib")

## **Model Validation**

In [4]:
cols_to_remove = ["parents_children","num_rooms","age_min_max","age_group_ord","passenger_class_ord"]
X_test_temp = X_test.copy()
X_test_temp.drop(columns=cols_to_remove, inplace=True)

y_predicted = logreg_model.predict(X_test_temp)

## **Evaluation Metrics**

In [5]:
test_conf_matrix = pd.DataFrame(
    confusion_matrix(y_test, y_predicted),
    index=["actual no", "actual yes"],
    columns=["predicted no", "predicted yes"],
)

test_conf_matrix

,predicted no,predicted yes
actual no,236,182
actual yes,0,0


In [6]:
metrics_dict = {
    "Accuracy": accuracy_score(y_test, y_predicted),
    "Precision": precision_score(y_test, y_predicted),
    "Recall": recall_score(y_test, y_predicted),
    "F1 Score": f1_score(y_test, y_predicted),
}

metrics_df = pd.DataFrame(
    metrics_dict.values(), index=metrics_dict.keys(), columns=["Value"]
)

metrics_df

c:\Users\mario\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Value
Accuracy,0.564593
Precision,0.000000
Recall,0.000000
F1 Score,0.000000


## **Model & Prediction Export**

In [7]:
os.chdir(MODELS_DIR)

kaggle_df = pd.DataFrame(y_predicted, index=X_test.index, columns=["Survived"])
kaggle_df.index.name = "PassengerId"
kaggle_df.replace({False: 0, True: 1}, inplace=True)
kaggle_df.to_csv("kaggle_submission.csv")
kaggle_df.head()

C:\Users\mario\AppData\Local\Temp\ipykernel_15432\2406585246.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  kaggle_df.replace({False: 0, True: 1}, inplace=True)


,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1
